# 2.1 - Prompts and Output Parsers 

## Setup

### Install dependencies

In [ ]:
%pip install python-dotenv~=1.0 docarray~=0.40.0 pydantic~=2.9 pypdf~=5.1 --upgrade --quiet
%pip install langchain~=0.3.7 langchain_openai~=0.2.6 langchain_community~=0.3.5 --upgrade --quiet

# If running locally, you can do this instead:
#%pip install -r ../requirements.txt

### Load environment variables

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# If running in Google Colab, you can use this code instead:
# from google.colab import userdata
# os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")

### Setup Models

In [ ]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
api_version = "2024-10-01-preview"
llm = AzureChatOpenAI(deployment_name="gpt-4o", temperature=0.0, api_version=api_version)
embedding_model = AzureOpenAIEmbeddings(model="text-embedding-3-large", api_version=api_version)


## Prompts

### Prompt template - for any type of LM (instruct or base) and simple use cases

In [ ]:
from langchain.prompts import PromptTemplate

template_string = ("Translate the text that is delimited by triple backticks into a style that "
                   "is {style}. text: ```{text}```")

prompt = PromptTemplate.from_template(template_string)
prompt.format(style="geeky", text="Hello, world!")

### Chat prompt template - for chat-based LLMs
Basically a chat prompt template is a list of messages, where each message in itself can be a template.

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_template = "Translate user input into a style that is {style}."

prompt_template_typed = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{input}"),
])

# Use can also use this shorthand to create the same prompt template    
prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("human", "{input}"),
])


In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
customer_style = "American English in a calm and respectful tone"

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = prompt_template.format_messages(
    style=customer_style,
    input=customer_email)

print(type(customer_messages))
print(type(customer_messages[0]))
print(type(customer_messages[1]))

In [ ]:
print(customer_messages[0])

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = llm.invoke(customer_messages)
print(customer_response.content)

#### Try another example

In [ ]:
customer_style = "The same style as the user"

customer_messages = prompt_template.format_messages(
    style=customer_style,
    input=customer_email)

customer_response = llm.invoke(customer_messages)
print(customer_response.content)

### MessagesPlaceholder

What if we wanted the user to pass in a list of messages that we would slot into a particular spot? This is how you use MessagesPlaceholder. A typical use case for this in when you need to include **_conversation history_** in the prompt.

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("history")
])

prompt_template.invoke({"history": [HumanMessage(content="hi!"), AIMessage(content="Sup fam?")]})

In [ ]:
# Alternative, shorthand syntax
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("placeholder", "{history}") # <-- This is the changed part
])

<br/>

----
<br/>


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

## The most common parser - String output parsing

In [ ]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()
llm.invoke("")

parsed_response = str_parser.invoke(customer_response)
print(parsed_response)

### Structured output parsing

In [ ]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable!"
}

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
response = llm.invoke(messages)
print(response.content)

In [ ]:
type(response.content)

In [ ]:
# You will get an error by running this line of code 
# because content is not a dictionary, but a string
response.content.get('gift') # This will raise an error - comment out to proceed

### Parse the LLM output string into a Python dictionary

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                  format_instructions=format_instructions)

In [ ]:
response = llm.invoke(messages)
print(response.content)

In [ ]:
print(messages[0].content)

In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('delivery_days')